Opening file

In [22]:
import pandas as pd

path='C:/Users/COMPUMARTS/Downloads/data_cleaning/Cafe data cleaning/cafe_sales.csv'
df=pd.read_csv(path)
# df.head(25)


Replace spaces in columns to avoid any errors

In [23]:
df.columns = df.columns.str.replace(' ', '_')
df.columns

Index(['Transaction_ID', 'Item', 'Quantity', 'Price_Per_Unit', 'Total_Spent',
       'Payment_Method', 'Location', 'Transaction_Date'],
      dtype='object')

Drop unnecessary columns or columns that hard to fill its null values

In [24]:
df=df.drop(columns='Transaction_Date')
df.isna().sum()

Transaction_ID       0
Item               333
Quantity           138
Price_Per_Unit     179
Total_Spent        173
Payment_Method    2579
Location          3265
dtype: int64

Now standardizing all null values

In [ ]:
df = df.replace({'UNKNOWN':pd.NA, 'NaN':pd.NA , 'ERROR':pd.NA})
df.head()

,Transaction_ID,Item,Quantity,Price_Per_Unit,Total_Spent,Payment_Method,Location
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store
2,TXN_4271903,Cookie,4,1.0,<NA>,Credit Card,In-store
3,TXN_7034554,Salad,2,5.0,10.0,<NA>,<NA>
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store


Fill the Quantity null values
    by dividing the Total_Spent in Price_Per_Unit
    to insure getting the right values

In [26]:
 #! we used pd.to_numeric instead of df.astype to avoid the problems that may happen cause of null values
df['Total_Spent'] = pd.to_numeric(df['Total_Spent'], errors='coerce')
df['Price_Per_Unit'] = pd.to_numeric(df['Price_Per_Unit'], errors='coerce')


df['Quantity'] = df.apply(# i here is a row in the data frame
    lambda i: i['Total_Spent']/i['Price_Per_Unit'] if pd.isna(i['Quantity']) else i['Quantity'],axis=1)
# df.head()

Here we try to fill up Total_Spent column after we filled quantity column

In [27]:
df['Quantity'] = df['Quantity'].astype('float')
df['Total_Spent']=df.apply(
    lambda i:i['Quantity']*i['Price_Per_Unit'] if pd.isna(i['Total_Spent']) else i['Total_Spent'],axis=1)
# df.tail(60)

Now we will try to fill Price_per_Unit after we filled Quantity and Total_Spent columns

In [28]:
df['Price_Per_Unit'] = df.apply(
	lambda row: row['Total_Spent']/row['Quantity'] if pd.isna(row['Price_Per_Unit']) else row['Price_Per_Unit'],
	axis=1
)
# df.tail(60)

In [29]:
df.isna().sum()

Transaction_ID       0
Item               969
Quantity            38
Price_Per_Unit      38
Total_Spent         40
Payment_Method    3178
Location          3961
dtype: int64

To Check why some values are null

In [30]:
null_index = df[df['Price_Per_Unit'].isna()].index
print(df.isnull().sum())
for i in null_index:
    print(df.loc[i, ['Price_Per_Unit','Quantity','Total_Spent']])


Transaction_ID       0
Item               969
Quantity            38
Price_Per_Unit      38
Total_Spent         40
Payment_Method    3178
Location          3961
dtype: int64
Price_Per_Unit    NaN
Quantity          3.0
Total_Spent       NaN
Name: 65, dtype: object
Price_Per_Unit     NaN
Quantity           NaN
Total_Spent       12.0
Name: 629, dtype: object
Price_Per_Unit     NaN
Quantity           NaN
Total_Spent       20.0
Name: 912, dtype: object
Price_Per_Unit    NaN
Quantity          NaN
Total_Spent       3.0
Name: 1008, dtype: object
Price_Per_Unit    NaN
Quantity          NaN
Total_Spent       6.0
Name: 1436, dtype: object
Price_Per_Unit     NaN
Quantity           NaN
Total_Spent       16.0
Name: 1482, dtype: object
Price_Per_Unit    NaN
Quantity          2.0
Total_Spent       NaN
Name: 1674, dtype: object
Price_Per_Unit    NaN
Quantity          4.0
Total_Spent       NaN
Name: 1761, dtype: object
Price_Per_Unit    NaN
Quantity          2.0
Total_Spent       NaN
Name: 2229, dtype: 

To delete the null values that we can't find it

In [32]:
df=df.dropna(subset=['Quantity'])
df=df.dropna(subset=['Price_Per_Unit'])
df=df.dropna(subset=['Total_Spent'])
# print(df.isnull().sum())
df.shape

(9942, 7)

Now replacing the null values in Payment_Method and Location with mode

In [33]:
P_mode=df['Payment_Method'].mode()[0]
df['Payment_Method'].fillna(P_mode,inplace=True)

L_mode=df['Location'].mode()[0]
df['Location'].fillna(L_mode,inplace=True)
print(df.isnull().sum())


Transaction_ID      0
Item              963
Quantity            0
Price_Per_Unit      0
Total_Spent         0
Payment_Method      0
Location            0
dtype: int64


C:\Users\COMPUMARTS\AppData\Local\Temp\ipykernel_14912\3132539126.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Payment_Method'].fillna(P_mode,inplace=True)
C:\Users\COMPUMARTS\AppData\Local\Temp\ipykernel_14912\3132539126.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

Now we will get the unique values in Item column to try to replace the null values in it

In [ ]:
unique_items = df['Item'].unique()
unique_prices = df['Price_Per_Unit'].unique()
print(unique_items)
print(unique_prices)
# df[['Item','Price_Per_Unit']].head()


In [34]:
 #! cause of we have two products with the same price we replaced it with the mode
df.loc[df['Price_Per_Unit']==1.0,'Item']='Cookie'
df.loc[df['Price_Per_Unit']==1.5,'Item']='Tea'
df.loc[df['Price_Per_Unit']==2.0,'Item']='Coffee'
df.loc[df['Price_Per_Unit']==3.0,'Item']=df['Item'].mode()[0]
df.loc[df['Price_Per_Unit']==4.0,'Item']='Smoothie'
df.loc[df['Price_Per_Unit']==5.0,'Item']='Salad'
print(df.isna().sum())

Transaction_ID    0
Item              0
Quantity          0
Price_Per_Unit    0
Total_Spent       0
Payment_Method    0
Location          0
dtype: int64


In [ ]:
print(df.isna().sum())

In [ ]:
df.to_csv('C:/Users/COMPUMARTS/Downloads/data_cleaning/Cafe data cleaning/clean_cafe_sales_saved.csv')


In [ ]:
df.head(20)